In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# update working directory
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom')

In [ ]:
!pip install tensorboardX
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlin

In [ ]:
import os
import time
import torch.multiprocessing
from copy import deepcopy
from tqdm import tqdm
import shutil

from parameters import Parameters
from train.trainer import Trainer
from deepfake_datasets.datasets import get_dataloader
from train.early_stopping import EarlyStopping
from train.validate import validate_detection, validate_fully_supervised_localization
from utils.utils import compute_mean_iou, compute_mean_ap, compute_mean_f1, compute_mean_acc_detection, compute_mean_ap_detection, compute_batch_iou, compute_batch_localization_f1, compute_batch_ap, compute_accuracy_detection, compute_average_precision_detection


In [ ]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
# new_dataset_name = 'ldm'

# new_drive_root_path = f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets/dolos_data/celebahq/fake/{new_dataset_name}' # drive root path
# new_content_root_path = f'/content/datasets/dolos_data/celebahq/fake/{new_dataset_name}' # content root path

# # move the dataset from drive to /content (SSD) for better performance in I/O
# shutil.copytree(new_drive_root_path, new_content_root_path, dirs_exist_ok=True)

new_dataset_name = 'train_all_4_datasets'

zip_path = f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets_zip/combined_training_for_ood/{new_dataset_name}.zip'
new_content_root_path = f'/content/datasets/dolos_data/celebahq/fake/'

os.makedirs(new_content_root_path, exist_ok=True)

# move the dataset from drive to /content (SSD) for better performance in I/O
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(new_content_root_path)

In [ ]:
# set parameters
params = Parameters()

# set experiment name
params.experiment_name = 'training_bce+dice_vit_conv12_modified_pos_weights'

# set backbone
params.arch = 'CLIP:ViT-L/14,RN50'

# create output dirs
params.create_output_dirs()

# uncomment lines below to set new datasets
# new_drive_root_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets/dolos_data/celebahq/'
# new_root_path = '/content/dataset'
# new_root_path = '..\\DeCLIP-main\\datasets\\dolos_data\\celebahq\\'
# new_root_path = 'D:\\Python\\DeCLIP-main\\datasets\\dolos_data\\celebahq\\'


# move the dataset from drive to /content for better performance in I/O
# shutil.copytree(new_drive_root_path, new_root_path, dirs_exist_ok=True)

new_root_path = '/content/datasets/dolos_data/celebahq/'
new_dataset_name = 'train_all_4_datasets'

params.update_dolos_data_paths(new_root_path, new_dataset_name)

# fix the backbone - to train only the decoder
params.fix_backbone = True

# set the feature layer and the decoder type
params.feature_layer = 'layer20'
params.decoder_type = 'conv-12'

# set the batch size and num threads
params.batch_size = 32
params.num_threads = 8

# set the number of epochs
params.num_iter = 30

# set loss type
params.loss_type = 'bce'

for key, value in params.__dict__.items():
    print(f"{key}: {value}")

experiment_name: training_bce+dice_vit_conv12_modified_pos_weights
task_type: fully_supervised_localization
data_label: train
arch: CLIP:ViT-L/14,RN50
fix_backbone: True
weight_decay: 0.0
batch_size: 32
num_threads: 8
init_type: normal
init_gain: 0.02
train_dataset: train_all_4_datasets
decoder_type: conv-12
feature_layer: layer20
early_stop_epochs: 5
optim: adam
beta1: 0.9
lr: 0.001
show_loss_freq: 50
num_iter: 30
data_root_path: /content/datasets/dolos_data/celebahq
train_fake_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/images/train
valid_fake_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/images/valid
test_fake_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/images/test
train_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/masks/train
valid_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/masks/valid
test_masks_ground_truth_path: /co

In [ ]:
# get the model, along with its trainer
model_trainer = Trainer(params)

# state_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/train_all_4/interrupted/best_localization_model_vit_conv20_iou_54.5470_epoch_22.pth', map_location='cpu')
# model_trainer.model.load_state_dict(state_dict['model'], strict=False)
# model_trainer.model = model_trainer.model.cuda()

100%|████████████████████████████████████████| 890M/890M [00:02<00:00, 347MiB/s]
100%|████████████████████████████████████████| 244M/244M [00:00<00:00, 342MiB/s]


In [ ]:
# get the dataloaders
train_loader = get_dataloader(params)

val_params = deepcopy(params)
val_params.data_label = 'valid'
val_loader = get_dataloader(val_params)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# start the training loop
# model_trainer.compute_pos_weight_per_dataset(train_loader)
# print('Pos weight for fake pixels', model_trainer.pos_weights)
early_stopping = EarlyStopping(patience=params.early_stop_epochs, verbose=True, delta=-0.001)
best_metric = 0.0 # iou for localization, ap for detection
print('Length of training set:', len(train_loader.dataset))
print('Length of validation set:', len(val_loader.dataset))
start_time = time.time()
for epoch in tqdm(range(params.num_iter)):
    print('Epoch:', epoch)

    epoch_loss = 0
    for data in train_loader:
        model_trainer.total_steps += 1

        model_trainer.set_input(data)
        model_trainer.optimize_parameters()

        if model_trainer.total_steps % params.show_loss_freq == 0:
            epoch_loss += model_trainer.loss.item()
            print(f'Train Loss: {model_trainer.loss.item():.4f} at step {model_trainer.total_steps} \t Iter time: {(time.time() - start_time) / model_trainer.total_steps:.2f}')


    epoch_loss /= len(train_loader.dataset)
    print(f'Average Train Loss: {epoch_loss:.4f}')

    # compute training metrics
    if params.task_type == 'fully_supervised_localization':
        compute_mean_iou(model_trainer.ious, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.ious = []

        compute_mean_f1(model_trainer.f1_best, model_trainer.f1_fixed, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.f1_best = []
        model_trainer.f1_fixed = []

        compute_mean_ap(model_trainer.ap, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.ap = []


    # validate the model
    print('Validation:')
    if params.task_type == 'fully_supervised_localization':
        ious, f1_best, f1_fixed, mean_ap, _ = validate_fully_supervised_localization(model_trainer.model, val_loader, params.train_dataset)

        # compute metrics
        mean_iou = compute_mean_iou(ious, verbose=True, extra_text=f'Validation at epoch {epoch} ')

        mean_f1_best, mean_f1_fixed = compute_mean_f1(f1_best, f1_fixed, verbose=True, extra_text=f'Validation at epoch {epoch} ')

        mean_ap = compute_mean_ap(mean_ap, verbose=True, extra_text=f'Validation at epoch {epoch} ')

        # save the model if the mean iou is improved
        if mean_iou > best_metric:
            best_metric = mean_iou
            model_trainer.save_model(f'best_localization_model_iou_{mean_iou:.4f}.pth')
            print(f'Best model saved at epoch {epoch}!')

        # check for early stopping
        early_stopping(mean_iou)

    # check if early stopping is triggered
    if early_stopping.early_stop:
        print("Early stopping triggered")
        continue_training = model_trainer.adjust_learning_rate()
        if continue_training:
            print("Continuing training with a learning rate reduced by a factor of 10")
            early_stopping = EarlyStopping(patience=params.early_stop_epochs, verbose=True, delta=-0.002) # adjust the delta only once, otherwise stop completely
        else:
            print(f"Early stopping training at epoch {epoch}")
            break
    print()
    torch.cuda.empty_cache()

Length of training set: 12000
Length of validation set: 1200


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0
Train Loss: 0.2833 at step 50 	 Iter time: 1.33
Train Loss: 0.2631 at step 100 	 Iter time: 1.30
Train Loss: 0.2590 at step 150 	 Iter time: 1.29
Train Loss: 0.2281 at step 200 	 Iter time: 1.29
Train Loss: 0.2181 at step 250 	 Iter time: 1.28
Train Loss: 0.1875 at step 300 	 Iter time: 1.28
Train Loss: 0.2578 at step 350 	 Iter time: 1.28
Average Train Loss: 0.0001
Train epoch 0 Mean IOU: 26.9
Train epoch 0 Mean F1 best: 0.6897
Train epoch 0 Mean F1 fixed: 0.4104
Train epoch 0 Mean AP: 0.6051
Validation:
Length of dataset:  1200



100%|██████████| 38/38 [00:23<00:00,  1.62it/s]


Validation at epoch 0 Mean IOU: 40.4
Validation at epoch 0 Mean F1 best: 0.6624
Validation at epoch 0 Mean F1 fixed: 0.5479
Validation at epoch 0 Mean AP: 0.6184


...

Validation at epoch 27 Mean IOU: 62.18
Validation at epoch 27 Mean F1 best: 0.811
Validation at epoch 27 Mean F1 fixed: 0.7327
Validation at epoch 27 Mean AP: 0.8169
Early Stopping counter: 3 out of 5

Epoch: 28
Train Loss: 0.0210 at step 10550 	 Iter time: 1.33
Train Loss: 0.0201 at step 10600 	 Iter time: 1.33
Train Loss: 0.0193 at step 10650 	 Iter time: 1.33
Train Loss: 0.0223 at step 10700 	 Iter time: 1.33
Train Loss: 0.0181 at step 10750 	 Iter time: 1.33
Train Loss: 0.0178 at step 10800 	 Iter time: 1.33
Train Loss: 0.0200 at step 10850 	 Iter time: 1.33
Average Train Loss: 0.0000
Train epoch 28 Mean IOU: 90.83
Train epoch 28 Mean F1 best: 0.9966
Train epoch 28 Mean F1 fixed: 0.9952
Train epoch 28 Mean AP: 0.9876
Validation:
Length of dataset:  1200



 97%|█████████▋| 29/30 [4:01:35<08:19, 499.33s/it]

Validation at epoch 28 Mean IOU: 62.84
Validation at epoch 28 Mean F1 best: 0.8056
Validation at epoch 28 Mean F1 fixed: 0.7308
Validation at epoch 28 Mean AP: 0.8161
Early Stopping counter: 4 out of 5

Epoch: 29
Train Loss: 0.0211 at step 10900 	 Iter time: 1.33
Train Loss: 0.0221 at step 10950 	 Iter time: 1.33
Train Loss: 0.0189 at step 11000 	 Iter time: 1.33
Train Loss: 0.0284 at step 11050 	 Iter time: 1.33
Train Loss: 0.0213 at step 11100 	 Iter time: 1.33
Train Loss: 0.0168 at step 11150 	 Iter time: 1.33
Train Loss: 0.0264 at step 11200 	 Iter time: 1.33
Train Loss: 0.0245 at step 11250 	 Iter time: 1.33
Average Train Loss: 0.0000
Train epoch 29 Mean IOU: 90.99
Train epoch 29 Mean F1 best: 0.9972
Train epoch 29 Mean F1 fixed: 0.9958
Train epoch 29 Mean AP: 0.9881
Validation:
Length of dataset:  1200



100%|██████████| 30/30 [4:09:54<00:00, 499.81s/it]

Validation at epoch 29 Mean IOU: 62.75
Validation at epoch 29 Mean F1 best: 0.8037
Validation at epoch 29 Mean F1 fixed: 0.7288
Validation at epoch 29 Mean AP: 0.8126
Early Stopping counter: 5 out of 5
Early stopping triggered
Continuing training with a learning rate reduced by a factor of 10



In [ ]:
from google.colab import files
model_dir = '/content/experiments/training_bce+dice_vit_conv12_modified_pos_weights/models/'
model_files = sorted(
    [f for f in os.listdir(model_dir) if f.endswith('.pth')]
)

best_model = model_files[-1]
best_model_path = os.path.join(model_dir, best_model)

files.download(best_model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>